This notebook outlines my process of building an auto-valuation-model that can predict the price of a property in Manhattan, New York, utilizing property features. Furthermore, I am going to utilize time series analysis to forecast future home values. This analysis will allow property owners to better understand the value of their property and make informed business decisions

link: https://www.nyc.gov/site/finance/property/glossary-property-sales.page

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
manhattan_data = pd.read_excel('data.xlsx')
time_series_data = pd.read_csv('data_monthly.csv')

In [4]:
# Display the first few rows
df = manhattan_data
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,377,10,NaN,C0,356 E 8 ST,NaN,...,3.0,0.0,3.0,1804.0,2928.0,1899.0,1,C0,2000000,2024-03-13
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,376,9,NaN,C1,234 E 7TH ST,NaN,...,20.0,0.0,20.0,2059.0,6490.0,1900.0,2,C1,5615000,2024-05-30
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,376,54,NaN,C4,"719 E 6TH STREET, 4C",NaN,...,20.0,0.0,20.0,2302.0,9750.0,1900.0,2,C4,168494,2024-05-28
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,38,NaN,C4,"21 AVENUE C, 2B",NaN,...,20.0,2.0,22.0,3315.0,17160.0,1900.0,2,C4,163587,2024-05-16
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385,38,NaN,C4,"21 AVENUE C, 4A",NaN,...,20.0,2.0,22.0,3315.0,17160.0,1900.0,2,C4,116848,2024-05-31


In [5]:
ts_df = time_series_data
ts_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,...,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30
0,102001,0,United States,country,NaN,170812.0,175500.0,177500.0,180000.0,185000.0,...,335000.0,345000.0,350000.0,360000.0,369000.0,360000.0,359000.0,350000.0,353187.0,350000.0
1,394913,1,"New York, NY",msa,NY,400000.0,390000.0,390000.0,393000.0,400000.0,...,570000.0,575000.0,590000.0,620000.0,645000.0,650000.0,644315.0,630000.0,625000.0,625000.0
2,753899,2,"Los Angeles, CA",msa,CA,470000.0,455000.0,457000.0,440000.0,435000.0,...,915000.0,916500.0,950000.0,970000.0,960000.0,951000.0,933750.0,915000.0,941000.0,922000.0
3,394463,3,"Chicago, IL",msa,IL,216000.0,220000.0,221000.0,227000.0,235000.0,...,285000.0,300000.0,315000.0,325000.0,337000.0,330000.0,320000.0,317900.0,310000.0,315000.0
4,394514,4,"Dallas, TX",msa,TX,138000.0,146000.0,144900.0,150000.0,156000.0,...,380000.0,390000.0,400000.0,405000.0,400000.0,396000.0,390000.0,380000.0,383000.0,385000.0
